<a href="https://colab.research.google.com/github/awesomeday3/Aww/blob/main/colab/Silly%20Tavern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI갤 실리 태번 실행기🤖 

✔ **확장 기능이 연결 안 되는 경우 (설치 과정에서 오류 난 것!)**

1. 코랩 창에서 오른쪽에 있는 연결 ▼ 클릭
2. 런타임 연결 해제 및 삭제 클릭
3. 세션 관리에서 코랩이 완전히 종료된 거 확인 후 재실행

In [ ]:
#@title Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start Silly Tavern below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title Silly Tavern
#@markdown **<font color = red>깡들 여기 눌러 (≖ ‸ ≖ ✿)**
#@markdown ###### ***옵션. NGROK 사용 ( 입력시 자동실행 )***
#@markdown - 아래 링크 Your Authtoken에서 토큰 발급받고 입력 후 실행 ㄱㄱ 로딩 더 빠름
#@markdown - https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_token = "" #@param {type:"string"}
#@markdown ---

ForceInitSteps = []
UseGoogleDrive = True #@param {type:"boolean"}
UseExtrasExtensions = True #@param {type:"boolean"}
#@markdown Enables hosting of extensions backend for SillyTavern Extras
extras_enable_captioning = True #@param {type:"boolean"}
#@markdown Loads the image captioning module
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
extras_enable_emotions = True #@param {type:"boolean"}
#@markdown Loads the sentiment classification model
Emotions_Model = "nateraw/bert-base-uncased-emotion" #@param ["nateraw/bert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
#@markdown * nateraw/bert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
#@markdown Loads the story summarization module
Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative


%cd /content

!cat .ii
!nvidia-smi

import os, subprocess, time, pathlib, json, base64, sys

if NGROK_token:
  !pip install pyngrok
  from pyngrok import ngrok
  os.environ['NGROK_token'] = NGROK_token
  !ngrok authtoken $NGROK_token
  !streamlit run app.py&>/dev/null&
  publ_url = ngrok.connect(addr = '5001')
else:
  publ_url = ''
  print("NGROK 미사용")

# ---
# Utils
class IncrementialInstall:
  def __init__(self, root = "/", tasks = [], force = []):
    self.tasks = tasks
    self.path = os.path.join(root, ".ii")
    self.completed = list(filter(lambda x: not x in force, self.__completed()))

  def __completed(self):
    try:
      with open(self.path) as f:
        return json.load(f)
    except:
      return []

  def addTask(self, name, func):
    self.tasks.append({"name": name, "func": func})

  def run(self):
    todo = list(filter(lambda x: not x["name"] in self.completed, self.tasks))
    try:
      for task in todo:
        task["func"]()
        self.completed.append(task["name"])
    finally:
      with open(self.path, "w") as f:
        json.dump(self.completed, f)

def create_paths(paths):
  for directory in paths:
    if not os.path.exists(directory):
      os.makedirs(directory)

def link(srcDir, destDir, files):
  '''
    Link source to dest copying dest to source if not present first
  '''
  for file in files:
    source = os.path.join(srcDir, file)
    dest = os.path.join(destDir, file)
    if not os.path.exists(source):
      !cp -r "$dest" "$source"
    !rm -rf "$dest"
    !ln -fs "$source" "$dest"

from google.colab import drive
if UseGoogleDrive:
  drive.mount("/content/drive/")
else:
  create_paths([
      "/content/drive/MyDrive"
  ])

ii = IncrementialInstall(force=ForceInitSteps)

# ---
# SillyTavern py modules
def cloneTavern():
  %cd /
  !git clone https://github.com/awesomeday3/SillyTavern
  %cd -
  !cp /SillyTavern/colab/*.py ./
ii.addTask("Clone SillyTavern", cloneTavern)
ii.run()

# ---
# nodejs
%cd /
def installNode():
  !npm install -g n
  !n 19
  !node --version
ii.addTask("Install node", installNode)


# ---
# SillyTavern extras
import globals
globals.extras_url = '(disabled)'
globals.params = []
globals.params.append('--cpu')
ExtrasModules = []

if (extras_enable_captioning):
  ExtrasModules.append('caption')
if (extras_enable_memory):
  ExtrasModules.append('summarize')
if (extras_enable_emotions):
  ExtrasModules.append('classify')

globals.params.append(f'--classification-model={Emotions_Model}')
globals.params.append(f'--summarization-model={Memory_Model}')
globals.params.append(f'--captioning-model={Captions_Model}')
globals.params.append(f'--enable-modules={",".join(ExtrasModules)}')


if UseExtrasExtensions:
    def cloneExtras():
        %cd /
        !git clone https://github.com/awesomeday3/SillyTavern-extras
    ii.addTask('clone extras', cloneExtras)

    def installRequirements():
        %cd /SillyTavern-extras
        !npm install -g localtunnel
        !pip install -r requirements.txt
        !pip install tensorflow==2.12
    ii.addTask('install requirements', installRequirements)

    from extras_server import runServer, extractUrl
    ii.addTask('run server', runServer)
    ii.addTask('extract extras URL', extractUrl)

%cd /SillyTavern

if UseGoogleDrive:
  %env googledrive=2

  def setupTavernPaths():
    %cd /SillyTavern
    tdrive = "/content/drive/MyDrive/SillyTavern"
    create_paths([
        tdrive,
        os.path.join("public", "groups"),
        os.path.join("public", "group chats")
    ])
    link(tdrive, "public", [
        "settings.json",
        "backgrounds",
        "characters",
        "chats",
        "User Avatars",
        "worlds",
        "group chats",
        "groups",
    ])
  ii.addTask("Setup Tavern Paths", setupTavernPaths)

def installTavernDependencies():
  %cd /SillyTavern
  !npm install
  !npm install -g localtunnel
  !npm install -g forever
  !pip install flask-cloudflared==0.0.10
ii.addTask("Install Tavern Dependencies", installTavernDependencies)
ii.run()

%env colaburl=$url
%env SILLY_TAVERN_PORT=5001
!sed -i 's/listen = true/listen = false/g' config.conf
!touch stdout.log stderr.log
!forever start -o stdout.log -e stderr.log server.js
print('###Extensions API LINK###', globals.extras_url,  "###SillyTavern LINK###", sep="\n")
from flask_cloudflared import _run_cloudflared
cloudflare = _run_cloudflared(5001)
print(cloudflare)
print(publ_url)
!tail -f stdout.log stderr.log